## 5.1 Initialization

In [68]:
import numpy as np
import numpy.testing as npt
from scipy.linalg import expm
import matplotlib.pyplot as plt
import scipy.spatial, scipy.linalg
from scipy.spatial.distance import pdist
import scipy.sparse.linalg
from scipy.linalg import eigh
import zipfile,io
import pandas as pd
import zipfile
import re
%matplotlib inline

(a) Load the sound files. Each of the N = 2 sources is sampled at at 8192 Hz and contains p = 18000 samples.

In [69]:
unzip = zipfile.ZipFile("sounds.zip", 'r')          
fl = unzip.namelist()                     
s1 = unzip.open(fl[0])
s2 = unzip.open(fl[1])

In [70]:
dat1 = pd.read_table(s1,header=None,delim_whitespace=True)
dat2 = pd.read_table(s2,header=None,delim_whitespace=True)

In [71]:
sig1 = np.array(dat1)
sig2 = np.array(dat2)
S= np.append(sig1 ,sig2,1)
S

array([[ -0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,  -1.58992990e-04],
       [ -3.77485200e-05,  -3.87545410e-03],
       ..., 
       [  4.35995410e-03,  -4.59092250e-03],
       [ -2.78395330e-03,  -3.24941920e-03],
       [  0.00000000e+00,  -0.00000000e+00]])

(b) Create a random (& invertible) N ×N mixing matrix A and mix the sources: x(α) = As(α)

In [72]:
n = 2
A = np.random.rand(n,n)
print(A)
detA =np.linalg.det(A)
while(detA<0.001):
    A = np.random.rand(n,n)
    detA =np.linalg.det(A)

[[ 0.70465486  0.18268301]
 [ 0.54662364  0.98093829]]


(c) Remove the temporal structure by permuting the columns of the N × p matrix X randomly

In [73]:
X = np.dot(A,S.T)
p = len(X[0])
rmidx = np.random.choice(p,p,replace=False)
XX=X[:,rmidx]

(d) Calculate the correlations between the sources and the mixtures: 

In [74]:
meanx = np.mean(XX,1)
means = np.mean(S,0)
C = np.dot(XX-meanx.reshape(2,1),(S-means))/p
sdx = np.std(XX, axis=1)
sds = np.std(S, axis=0)
print(sdx)
print(sds)
matsdx = (np.ones((2,1))*sdx).T
matsds = (np.ones((2,1))*sds)
print(matsdx)
print(matsds)
corr = C/matsdx/matsds 
print(C)
corr

[ 0.72712849  1.12219145]
[ 0.99854594  0.99885435]
[[ 0.72712849  0.72712849]
 [ 1.12219145  1.12219145]]
[[ 0.99854594  0.99885435]
 [ 0.99854594  0.99885435]]
[[-0.00374007  0.00054235]
 [-0.01060225 -0.00450818]]


array([[-0.00515111,  0.00074674],
       [-0.00946157, -0.00402191]])

(e) Center the data to zero mean.


In [75]:
SS = S -np.mean(S,0)
XX.shape

(2, 18000)

(f) Initialize the unmixing matrix W with random values.

In [117]:
W= np.random.rand(n,n)
detW =np.linalg.det(W)
while(detW<0.001):
    W = np.random.rand(n,n)
    detW =np.linalg.det(W)
print(W)

[[ 0.47726395  0.59458109]
 [ 0.4096291   0.86904217]]


## 5.2 Optimization
Implement a matrix version of the ICA online learning algorithm. For f^use the logistic function(see lecture notes). This should reduce your code for this part to a few lines with one loop (over the samples). Implement two variants of this learning algorithm:

(a) Compute the update matrix ∆W using the “regular” gradient.

In [118]:
def sigmoid(y):
    return 1/(1+np.exp(-y))

In [131]:
dw = np.array(W)
ddw = np.empty((n,n))
eps=0.4
#“regular” gradient batch
for i in range(p):
    invW= np.linalg.solve(W,np.eye(n))
    y = np.dot(W,XX[:,i].reshape(n,1))
    sig = 1 -2*sigmoid(y)
    dw =(invW +sig*XX[:,i].reshape(n,1)*np.ones((1,n))).T
    ddw=ddw+dw
dw=eps*ddw/p

In [132]:
dw

array([[ 1.93847385, -1.15269632],
       [-1.48111122,  0.91942584]])

In [133]:
W +dw

array([[ 2.4157378 , -0.55811523],
       [-1.07148213,  1.78846801]])

In [134]:
np.linalg.inv(A)

array([[ 1.65877219, -0.30891801],
       [-0.92434366,  1.19157535]])

In [138]:
dw = np.array(W)
ww = np.array(W)
ddw = np.empty((n,n))
eps=0.0001
#“regular” gradient online
for i in range(p):
    invW= np.linalg.solve(ww,np.eye(n))
    y = np.dot(ww,XX[:,i].reshape(n,1))
    sig = 1 -2*sigmoid(y)
    dw =(invW +sig*XX[:,i].reshape(n,1)*np.ones((1,n))).T
    ww=ww+eps*dw

In [139]:
ww

array([[ 1.69690105, -0.39050374],
       [-0.27172327,  1.45474326]])

In [140]:
np.linalg.inv(A)

array([[ 1.65877219, -0.30891801],
       [-0.92434366,  1.19157535]])

(b) Compute the update matrix ∆W using the natural gradient as described in the lecture notes.

In [221]:
dw = np.array(W)
ww = np.array(W)
eps=0.0001
# natural gradient  
for i in range(p):
    y = np.dot(ww,XX[:,i].reshape(n,1))
    sig = 1 -2*sigmoid(y)
    sig=sig*np.ones((2,2))
    wx= y.reshape(n,1,1)
    dw = np.diag(ww.sum(0))+sig*(ww.reshape(n,1,n)*np.ones((n,n,1))*wx).sum(0)
    ww=ww+eps*dw

In [222]:
ww

array([[ 1.88030179, -0.31134069],
       [-0.24726632,  1.51353682]])

In [223]:
dw

array([[ 0.92789956, -1.00368882],
       [-1.01120154, -0.23700474]])

In [224]:
np.linalg.inv(A)

array([[ 1.65877219, -0.30891801],
       [-0.92434366,  1.19157535]])

(c) Choose a suitable learning rate " and apply both versions to the data to unmix the sources.

With our experiment by online lerning of regular gradient and natural gradient we found that best eps is

eps = 0.0001

## 5.3 Results